# Playing decrypto with AI

## Importing librairies

In [51]:
import numpy as np
import pickle
from scipy import spatial
from random import sample

dim = 300 #Dimension of the embeddings

## The Model

Saving pre-trained word embeddings dict to pickle format

In [52]:
# embeddings_dict = {}
# with open("glove.6B.300d.txt", 'r',encoding="utf-8") as f:
#     for line in f:
#         values = line.split()
#         word = values[0]
#         vector = np.asarray(values[1:], "float32")
#         embeddings_dict[word] = vector
# with open('data_300.pickle', 'wb') as handle:
#     pickle.dump(embeddings_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

Loading pre-trained word embeddings from the pickle file

In [53]:
with open('data_300.pickle', 'rb') as fp:
    embeddings_dict = pickle.load(fp)

Functions

In [54]:
def get_centers(hints):
    centers = []
    for hints_list in hints:
        embeddings = []
        if hints_list == []:
            centers.append(np.zeros(dim))
        else :
            for word in hints_list:
                embeddings.append(embeddings_dict[word])
            centers.append(np.sum(embeddings,axis=0)/len(embeddings))
    return centers
    
def guess_word_1(hints,new_hint):
    embedding = embeddings_dict[new_hint]
    D = []
    for word_list in hints:
        dist = 0
        for word in word_list :
            dist += spatial.distance.cosine(embeddings_dict[word], embedding)
        D.append(dist)
    return np.argmin(D)

def guess_word_2(hints_centers,new_hint):
    embedding = embeddings_dict[new_hint]
    D = []
    for rep in hints_centers:
        dist = spatial.distance.cosine(rep, embedding)
        D.append(dist)
    return np.argmin(D)

Finding the closest group of word to the new hint 

In [55]:
keys = ['fruits','clothes','technology','sports']

hints = [['orange'],
         ['pants'],
         ['phone'],
         ['football','run']]

new_hint = 'golf'

In [56]:
print("New hint : ",new_hint)
print("Sum of errors : ",keys[guess_word_1(hints,new_hint)])
print("Words centers : ",keys[guess_word_2(get_centers(hints),new_hint)])

New hint :  golf
Sum of errors :  clothes
Words centers :  sports


## The game

Generating four random words from the words list

In [86]:
with open('wordlist.txt', 'r') as file:
    content = file.read().splitlines()


randomlist = sample(range(len(content)), 4)
keys = [content[i] for i in randomlist]

print(keys)

['ADVICE', 'DENTIST', 'HAT', 'ENGLAND']


In [87]:
round = 0
hints = [[],
         [],
         [],
         []]

while True:
    code = sample(range(0,4), 3)
    print("The code is : ",code)
    print("The keys are : ",keys)

    words = input("Enter the three words separated by a space : ").split()

    while not all(word in embeddings_dict.keys() for word in words):
        words = input("RE-enter the three words separated by a space : ").split()
    if round == 0:
        guesses = sample(range(0,4), 3)
    else:
        guesses = []
        for word in words:
            guesses.append(guess_word_2(get_centers(hints),word))

    if code == guesses:
        print("HIT")
        break
    else:
        print("MISS Guessed : ",guesses)

    for i in range(3):
        hints[code[i]].append(words[i])

    print(hints)
    
    round += 1

The code is :  [2, 3, 0]
The keys are :  ['ADVICE', 'DENTIST', 'HAT', 'ENGLAND']
MISS Guessed :  [3, 0, 2]
[['good'], [], ['head'], ['europe']]
The code is :  [0, 3, 1]
The keys are :  ['ADVICE', 'DENTIST', 'HAT', 'ENGLAND']
MISS Guessed :  [1, 1, 1]
[['good', 'opinion'], ['pain'], ['head'], ['europe', 'beer']]
The code is :  [1, 2, 3]
The keys are :  ['ADVICE', 'DENTIST', 'HAT', 'ENGLAND']
MISS Guessed :  [2, 2, 3]
[['good', 'opinion'], ['pain', 'mouth'], ['head', 'hair'], ['europe', 'beer', 'uk']]
The code is :  [3, 1, 2]
The keys are :  ['ADVICE', 'DENTIST', 'HAT', 'ENGLAND']
MISS Guessed :  [2, 1, 2]
[['good', 'opinion'], ['pain', 'mouth', 'doctor'], ['head', 'hair', 'clothes'], ['europe', 'beer', 'uk', 'elizabeth']]
The code is :  [0, 1, 3]
The keys are :  ['ADVICE', 'DENTIST', 'HAT', 'ENGLAND']
HIT


In [91]:
print("Game finished after {} rounds".format(round))
print("The hints given for the keys {} are :\n{}".format(keys,hints))

Game finished after 4 rounds
the hints given for the keys ['ADVICE', 'DENTIST', 'HAT', 'ENGLAND'] are :
[['good', 'opinion'], ['pain', 'mouth', 'doctor'], ['head', 'hair', 'clothes'], ['europe', 'beer', 'uk', 'elizabeth']]
